# TMBA Practice: Camera Projections

Alberto Rota: alberto1.rota@polimi.it
***

## Camera Calibration
After printing the chequered board, we calibrate the camera. GOAL: To obtain the intrinsics

In [ ]:
import numpy as np
import yaml
import cv2
import sys

# Print Versions
print("Numpy Version: ", np.__version__)
print("OpenCV Version: ", cv2.__version__)
print("Yaml Version: ", yaml.__version__)
print("Python Version: ", sys.version)

Put `CAMERA=0` if you want to use the webcam, or `CAMERA=1` if you have an external USB camera

In [3]:
CAMERA = 1

- Press Space to acquire the corner points on the chequered board
- Press C tocalibrate the camera
- Press ESC to exit  (NOTE: Closing with camera window with the mouse will probably give problems)

In [ ]:
n_row=7
n_col=7
n_min_img = 10 # img needed for calibration
criteria = (cv2.TERM_CRITERIA_EPS + cv2.TERM_CRITERIA_MAX_ITER, 30, 0.001) # termination criteria
corner_accuracy = (11,11)
_____ = 0
result_file = "./calibration.yaml"

# prepare object points, like (0,0,0), (1,0,0), (2,0,0) ....,(n_row-1,n_col-1,0)
objp = np.zeros((n_row*n_col,3), np.float32)
objp[:,:2] = np.mgrid[0:n_row,0:n_col].T.reshape(-1,2)

# Intialize camera and window
camera = cv2.VideoCapture(CAMERA) #Supposed to be the only camera
if not camera.isOpened():
    print("Camera not found!")
    quit()
width = int(camera.get(cv2.CAP_PROP_FRAME_WIDTH))  
height = int(camera.get(cv2.CAP_PROP_FRAME_HEIGHT))
cv2.namedWindow("Calibration")


Initialization = True

while True:    
    if Initialization:
        # print("Initialize data structures ..")
        objpoints = [] # 3d point in real world space
        imgpoints = [] # 2d points in image plane.
        n_img = 0
        Initialization = False
        tot_error=0
    
    # Read from camera and display on windows
    ret, img = camera.read()
    cv2.imshow("Calibration", img)
    if not ret:
        # print("Cannot read camera frame")
        camera.release()        
        cv2.destroyAllWindows()
        break
    
    # Wait for instruction 
    k = cv2.waitKey(50) 
   
    # SPACE pressed to take picture
    if k%256 == 32:   
        # print("Adding image for calibration...")
        imgGray = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)

        # Find the chess board corners
        ret, corners = cv2.findChessboardCorners(imgGray, (n_row,n_col),None)

        # If found, add object points, image points (after refining them)
        if not ret:
            # print("Cannot found Chessboard corners!")
            continue
        else: 
            n_img +=1
            print(f"[{n_img}/{n_min_img}] Chessboard corners successfully found.")
            objpoints.append(objp)
            corners2 = cv2.cornerSubPix(imgGray,corners,corner_accuracy,(-1,-1),criteria)
            imgpoints.append(corners2)

            # Draw and display the corners
            imgAugmnt = cv2.drawChessboardCorners(img, (n_row,n_col), corners2,ret)
            cv2.imshow('Calibration',imgAugmnt) 
            cv2.waitKey(500)        
                
    # "c" pressed to compute calibration        
    elif k%256 == 99:        
        if n_img <= n_min_img:
            print("Only ", n_img , " captured, ",  " at least ", n_min_img , " images are needed")
        
        else:
            print("Computing calibration ...")
            ret, INTRINSICS, DISTORTION, rvecs, tvecs = cv2.calibrateCamera(objpoints, imgpoints, (width,height),None,None)
            
            if not ret:
                print("Cannot compute calibration!")
            
            else:
                # print("Camera calibration successfully computed")
                # Compute reprojection errors
                print("Camera matrix: \n", INTRINSICS)
                print("Distortion coeffs: \n ", DISTORTION)
                
                # Saving calibration matrix
                print("Saving camera matrix .. in ",result_file)
                data={"camera_matrix": INTRINSICS.tolist(), "dist_coeff": DISTORTION.tolist()}
                with open(result_file, "w") as f:
                    yaml.dump(data, f, default_flow_style=False)
                
    # ESC pressed to quit
    elif k%256 == 27:
            print("Escape hit, closing...")
            camera.release()        
            cv2.destroyAllWindows()
            break
    # "r" pressed to reset
    elif k%256 ==114: 
         print("Reset program...")
         Initialization = True        


***

# World-to-Pixel Transformation


The camera is calibrated. Let's check that the mapping between the 3D world and the 2D pixel coordinates is correct.

In [ ]:
import numpy as np
import yaml
import cv2
import sys
from rich import print

CAMERA = 1

with open("calibration.yaml", 'r') as stream:
    try:
        data = yaml.safe_load(stream)
        INTRINSICS = np.array(data['camera_matrix'])
        DISTORTION = np.array(data['dist_coeff'])
    except yaml.YAMLError as exc:
        print(exc)
        
print("Projection Matrix: \n", INTRINSICS)

# TODO: Put the 3D coordinates here
WORLD_X = _____
WORLD_Y = _____ - 20.0 # The camera il 20mm above the ground
WORLD_Z = _____

# The most important function in this code
camera_point = cv2.projectPoints(np.array([[WORLD_X,WORLD_Y,WORLD_Z]]), np.identity(3), np.zeros(3), INTRINSICS, None)[0][0][0]
#--------------------------------------------------------------------------------------------------------------------------------
print(camera_point.round())

def click_event(event, x, y, flags, params):
    # y+=240
    if event == cv2.EVENT_LBUTTONDOWN:
        print(f' [{  width-x},{height-y}]')
      
        # cv2.putText(image, f'({  width-x},{height-y})',(x,y),
        # cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2)
      
        # cv2.circle(image, (x,y), 3, (0,255,255), -1)
        # cv2.waitKey(5000)

camera = cv2.VideoCapture(CAMERA) 
width = int(camera.get(cv2.CAP_PROP_FRAME_WIDTH))  
height = int(camera.get(cv2.CAP_PROP_FRAME_HEIGHT))
if not camera.isOpened():
    print("Camera not found!")
    quit()
width = int(camera.get(cv2.CAP_PROP_FRAME_WIDTH))  
height = int(camera.get(cv2.CAP_PROP_FRAME_HEIGHT))
cv2.namedWindow("Transformation")

Initialization = True
DISTORTED = False

cv2.setMouseCallback('Transformation', click_event)

while True:    

    ret, image = camera.read()
    k = cv2.waitKey(10)
    cv2.imshow("Transformation", image)
    
    if not ret:
        print("Cannot read camera frame")
        camera.release()        
        cv2.destroyAllWindows() 
        break

    # Esc to quit
    if k%256 == 27:
        camera.release()        
        cv2.destroyAllWindows()
        break
        


***

## Transformations with Estrinsics Parameters

Transforming the points, but now the camera is not at the world origin anymore

In [ ]:
import numpy as np
import yaml
import cv2
import sys
# from rich import print

CAMERA = 1

with open("calibration.yaml", 'r') as stream:
    try:
        data = yaml.safe_load(stream)
        INTRINSICS = np.array(data['camera_matrix'])
        DISTORTION = np.array(data['dist_coeff'])
    except yaml.YAMLError as exc:
        print(exc)
        
# TODO: Add the camera motion (remember the .0)
CAMERA_X = _____
CAMERA_Y = _____ 
CAMERA_Z = _____

ESTRINSICS = np.eye(4)
ESTRINSICS[0,-1] = CAMERA_X*-1
ESTRINSICS[1,-1] = CAMERA_Y*-1
ESTRINSICS[2,-1] = CAMERA_Z*-1

INTRINSICS_H = np.eye(4)
INTRINSICS_H[:3,:3] = INTRINSICS
INTR_ESTR = np.matmul(ESTRINSICS,INTRINSICS_H)

print("Intrinsic Matrix: \n", INTRINSICS_H)
print("Estrinsics Matrix: \n", ESTRINSICS)
print("Projection Matrix: \n", INTR_ESTR)

# TODO: Put the 3D coordinates here (remember the .0)
WORLD_X = _____
WORLD_Y = _____ - 20.0
WORLD_Z = _____

camera_point = cv2.projectPoints(np.array([[WORLD_X,WORLD_Y,WORLD_Z]]), np.identity(3), np.array([-CAMERA_X,-CAMERA_Y,-CAMERA_Z]), INTRINSICS , None)[0][0][0]
# print("\nPoint in camera: \nD ",camera_point.round())
# TODO: Remove this comments
camera_point_und = cv2.projectPoints(np.array([[WORLD_X,WORLD_Y,WORLD_Z]]), np.identity(3), np.array([-CAMERA_X,-CAMERA_Y,-CAMERA_Z]), INTRINSICS , DISTORTION)[0][0][0]
print("\nPoint in camera: \nD ",camera_point.round(), " UND",camera_point_und.round())


def click_event(event, x, y, flags, params):
    # y+=240
    if event == cv2.EVENT_LBUTTONDOWN:
        print(f' [{  width-x},{height-y}]')
      
    #   # put coordinates as text on the image
    #     cv2.putText(image, f'({x},{y})',(x,y),
    #     cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2)
      
    # #   # draw point on the image
    #     cv2.circle(image, (x,y), 3, (0,255,255), -1)
    #     cv2.waitKey(1000)

camera = cv2.VideoCapture(CAMERA)
width = int(camera.get(cv2.CAP_PROP_FRAME_WIDTH))  
height = int(camera.get(cv2.CAP_PROP_FRAME_HEIGHT))
if not camera.isOpened():
    print("Camera not found!")
    quit()
width = int(camera.get(cv2.CAP_PROP_FRAME_WIDTH))  
height = int(camera.get(cv2.CAP_PROP_FRAME_HEIGHT))
cv2.namedWindow("Transformation with Estrinsics")

cv2.setMouseCallback('Transformation with Estrinsics', click_event)

while True:    
    
    # Read from camera and display on windows
    ret, image = camera.read()
    k = cv2.waitKey(10)
    cv2.imshow("Transformation with Estrinsics", image)
    
    if not ret:
        
        print("Cannot read camera frame")
        camera.release()        
        cv2.destroyAllWindows() 
        break

    # Esc to quit
    if k%256 == 27:
        camera.release()        
        cv2.destroyAllWindows()
        break
        


***